
### Project: Image Captioning


In this notebook, we will train our CNN-RNN model. The notebook is composed of the following steps:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, we will setup the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

We refer [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  

### CNN-RNN architecture and hyperparameters

I use pre-trained Resnet-50 model that has 1000 FC size. 
I have chosen 512 as Embed size and also as Hidden size for training based on the paper provided above. <br>
(https://arxiv.org/pdf/1411.4555.pdf). On this paper it mentioned that they started with 512 for Embedding size and the LSTM memory size. And a result of training on this project with the sizes seemed also quite decent.<br>
The batch size was explained on Hyperparameters session, which said the bigger batch size, the better computational efficiency. But 256 is threshold; If the size exceeded that, a result goes worse. <br>
I started with vocab_threshold=5, but it seems like I have not enough data to train. <br>
So I changed it vocab_threshold to 4 and I could get the better result. <br>

For using a pre-trained model with 244x244x3 images, I chose normalized values mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225].(Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)) <br>
There are also flipping and resizing to 244x244 size for making this model to learn more effectively on CNN training phase. <br>
(Resizing and Flipping work great ecspecially when train data is not enough to train. It could make a model to see vary data for tuning parameters and also prevent overfitting) <br>
After I trained this model, I realized the transfrom was great enough.
Since I use the pre-trained model, I want to only train embed parameters on an encoder side,
I chose """encoder.embed.parameters()""" But for a decode side, we need to train all layers on a decoder side. <br>
so, """decoder.parameters()""" was a decent choice I think.

For the optimizer, At first, I started with SGD and changed lr multiple times. <br>
But it seems like finding a great lr for SGD was difficult in the case. <br>
(lr=0.1 seems not learing, lr=0.01|0.001 seems learning quite slowly) <br>
So I changed it to Adam which is explained as one of Adaptive Learning Optimizers <br>
on Hyperparameters lesson. I think Adam seems working fine on this training phase.

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
!pip install nltk
import nltk
nltk.download('punkt')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## Select appropriate values for the Python variables below.
batch_size = 256            ## batch size
vocab_threshold = 4        ## minimum word count threshold
vocab_from_file = False     ## if True, load existing vocab file
embed_size = 512           ## dimensionality of image and word embeddings
hidden_size = 512          ## number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# Specify the learnable parameters of the model.
# list(model_name.layer_name.parameters())
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 

# Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001)
#optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
loading annotations into memory...
Done (t=0.94s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...
Done (t=0.85s)
creating index...


  0%|          | 1319/414113 [00:00<01:02, 6635.80it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:02<00:00, 6620.38it/s]


<a id='step2'></a>
## Step 2: Train your Model


In [ ]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

num_epochs = 3

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

batch_size:  256
embed_size:  512
Epoch [1/3], Step [1/1618], Loss: 2.8392, Perplexity: 17.1028batch_size:  256
embed_size:  512
Epoch [1/3], Step [2/1618], Loss: 2.8776, Perplexity: 17.7719batch_size:  256
embed_size:  512
Epoch [1/3], Step [3/1618], Loss: 3.0990, Perplexity: 22.1752batch_size:  256
embed_size:  512
Epoch [1/3], Step [4/1618], Loss: 2.9265, Perplexity: 18.6625batch_size:  256
embed_size:  512
Epoch [1/3], Step [5/1618], Loss: 3.0549, Perplexity: 21.2189batch_size:  256
embed_size:  512
Epoch [1/3], Step [6/1618], Loss: 2.7600, Perplexity: 15.8003batch_size:  256
embed_size:  512
Epoch [1/3], Step [7/1618], Loss: 2.9878, Perplexity: 19.8416batch_size:  256
embed_size:  512
Epoch [1/3], Step [8/1618], Loss: 2.8919, Perplexity: 18.0281batch_size:  256
embed_size:  512
Epoch [1/3], Step [9/1618], Loss: 2.9113, Perplexity: 18.3814batch_size:  256
embed_size:  512
Epoch [1/3], Step [10/1618], Loss: 2.8511, Perplexity: 17.3071batch_size:  256
embed_size:  512
Epoch [1/3], St

Epoch [1/3], Step [170/1618], Loss: 2.5998, Perplexity: 13.4614batch_size:  256
embed_size:  512
Epoch [1/3], Step [171/1618], Loss: 2.7160, Perplexity: 15.1194batch_size:  256
embed_size:  512
Epoch [1/3], Step [172/1618], Loss: 3.0412, Perplexity: 20.9295batch_size:  256
embed_size:  512
Epoch [1/3], Step [173/1618], Loss: 2.7172, Perplexity: 15.1382batch_size:  256
embed_size:  512
Epoch [1/3], Step [174/1618], Loss: 2.8674, Perplexity: 17.5920batch_size:  256
embed_size:  512
Epoch [1/3], Step [175/1618], Loss: 2.5690, Perplexity: 13.0522batch_size:  256
embed_size:  512
Epoch [1/3], Step [176/1618], Loss: 2.8259, Perplexity: 16.8764batch_size:  256
embed_size:  512
Epoch [1/3], Step [177/1618], Loss: 2.8394, Perplexity: 17.1063batch_size:  256
embed_size:  512
Epoch [1/3], Step [178/1618], Loss: 3.0476, Perplexity: 21.0640batch_size:  256
embed_size:  512
Epoch [1/3], Step [179/1618], Loss: 2.7952, Perplexity: 16.3659batch_size:  256
embed_size:  512
Epoch [1/3], Step [180/1618], 

Epoch [1/3], Step [338/1618], Loss: 2.5770, Perplexity: 13.1576batch_size:  256
embed_size:  512
Epoch [1/3], Step [339/1618], Loss: 2.7205, Perplexity: 15.1886batch_size:  256
embed_size:  512
Epoch [1/3], Step [340/1618], Loss: 2.5802, Perplexity: 13.1998batch_size:  256
embed_size:  512
Epoch [1/3], Step [341/1618], Loss: 2.5499, Perplexity: 12.8063batch_size:  256
embed_size:  512
Epoch [1/3], Step [342/1618], Loss: 2.7889, Perplexity: 16.2635batch_size:  256
embed_size:  512
Epoch [1/3], Step [343/1618], Loss: 2.7216, Perplexity: 15.2048batch_size:  256
embed_size:  512
Epoch [1/3], Step [344/1618], Loss: 2.5232, Perplexity: 12.4681batch_size:  256
embed_size:  512
Epoch [1/3], Step [345/1618], Loss: 2.8939, Perplexity: 18.0644batch_size:  256
embed_size:  512
Epoch [1/3], Step [346/1618], Loss: 2.6795, Perplexity: 14.5784batch_size:  256
embed_size:  512
Epoch [1/3], Step [347/1618], Loss: 2.4677, Perplexity: 11.7955batch_size:  256
embed_size:  512
Epoch [1/3], Step [348/1618], 

Epoch [1/3], Step [506/1618], Loss: 2.5130, Perplexity: 12.3419batch_size:  256
embed_size:  512
Epoch [1/3], Step [507/1618], Loss: 2.4750, Perplexity: 11.8812batch_size:  256
embed_size:  512
Epoch [1/3], Step [508/1618], Loss: 2.3407, Perplexity: 10.3884batch_size:  256
embed_size:  512
Epoch [1/3], Step [509/1618], Loss: 2.5528, Perplexity: 12.8431batch_size:  256
embed_size:  512
Epoch [1/3], Step [510/1618], Loss: 2.7473, Perplexity: 15.6001batch_size:  256
embed_size:  512
Epoch [1/3], Step [511/1618], Loss: 2.5466, Perplexity: 12.7631batch_size:  256
embed_size:  512
Epoch [1/3], Step [512/1618], Loss: 3.0685, Perplexity: 21.5090batch_size:  256
embed_size:  512
Epoch [1/3], Step [513/1618], Loss: 3.1762, Perplexity: 23.9545batch_size:  256
embed_size:  512
Epoch [1/3], Step [514/1618], Loss: 2.4592, Perplexity: 11.6950batch_size:  256
embed_size:  512
Epoch [1/3], Step [515/1618], Loss: 2.4762, Perplexity: 11.8958batch_size:  256
embed_size:  512
Epoch [1/3], Step [516/1618], 

Epoch [1/3], Step [674/1618], Loss: 2.3518, Perplexity: 10.5040batch_size:  256
embed_size:  512
Epoch [1/3], Step [675/1618], Loss: 2.3039, Perplexity: 10.0137batch_size:  256
embed_size:  512
Epoch [1/3], Step [676/1618], Loss: 2.6635, Perplexity: 14.3467batch_size:  256
embed_size:  512
Epoch [1/3], Step [677/1618], Loss: 2.6308, Perplexity: 13.8852batch_size:  256
embed_size:  512
Epoch [1/3], Step [678/1618], Loss: 2.3937, Perplexity: 10.9535batch_size:  256
embed_size:  512
Epoch [1/3], Step [679/1618], Loss: 2.3849, Perplexity: 10.8576batch_size:  256
embed_size:  512
Epoch [1/3], Step [680/1618], Loss: 2.3116, Perplexity: 10.0908batch_size:  256
embed_size:  512
Epoch [1/3], Step [681/1618], Loss: 3.3673, Perplexity: 28.9989batch_size:  256
embed_size:  512
Epoch [1/3], Step [682/1618], Loss: 2.5666, Perplexity: 13.0213batch_size:  256
embed_size:  512
Epoch [1/3], Step [683/1618], Loss: 2.4342, Perplexity: 11.4064batch_size:  256
embed_size:  512
Epoch [1/3], Step [684/1618], 

...

Epoch [2/3], Step [918/1618], Loss: 2.0001, Perplexity: 7.3901batch_size:  256
embed_size:  512
Epoch [2/3], Step [919/1618], Loss: 2.0238, Perplexity: 7.5668batch_size:  256
embed_size:  512
Epoch [2/3], Step [920/1618], Loss: 2.2159, Perplexity: 9.1701batch_size:  256
embed_size:  512
Epoch [2/3], Step [921/1618], Loss: 2.1508, Perplexity: 8.5921batch_size:  256
embed_size:  512
Epoch [2/3], Step [922/1618], Loss: 2.1834, Perplexity: 8.8766batch_size:  256
embed_size:  512
Epoch [2/3], Step [923/1618], Loss: 2.2278, Perplexity: 9.2790batch_size:  256
embed_size:  512
Epoch [2/3], Step [924/1618], Loss: 2.0089, Perplexity: 7.4549batch_size:  256
embed_size:  512
Epoch [2/3], Step [925/1618], Loss: 2.3831, Perplexity: 10.8384batch_size:  256
embed_size:  512
Epoch [2/3], Step [926/1618], Loss: 2.0779, Perplexity: 7.9881batch_size:  256
embed_size:  512
Epoch [2/3], Step [927/1618], Loss: 2.2951, Perplexity: 9.9252batch_size:  256
embed_size:  512
Epoch [2/3], Step [928/1618], Loss: 2.4

Epoch [2/3], Step [1087/1618], Loss: 2.4584, Perplexity: 11.6865batch_size:  256
embed_size:  512
Epoch [2/3], Step [1088/1618], Loss: 2.0819, Perplexity: 8.0199batch_size:  256
embed_size:  512
Epoch [2/3], Step [1089/1618], Loss: 2.1006, Perplexity: 8.1711batch_size:  256
embed_size:  512
Epoch [2/3], Step [1090/1618], Loss: 2.0127, Perplexity: 7.4836batch_size:  256
embed_size:  512
Epoch [2/3], Step [1091/1618], Loss: 2.0424, Perplexity: 7.7093batch_size:  256
embed_size:  512
Epoch [2/3], Step [1092/1618], Loss: 2.0588, Perplexity: 7.8366batch_size:  256
embed_size:  512
Epoch [2/3], Step [1093/1618], Loss: 1.9025, Perplexity: 6.7027batch_size:  256
embed_size:  512
Epoch [2/3], Step [1094/1618], Loss: 2.1904, Perplexity: 8.9384batch_size:  256
embed_size:  512
Epoch [2/3], Step [1095/1618], Loss: 2.2113, Perplexity: 9.1278batch_size:  256
embed_size:  512
Epoch [2/3], Step [1096/1618], Loss: 2.1012, Perplexity: 8.1759batch_size:  256
embed_size:  512
Epoch [2/3], Step [1097/1618]

Epoch [2/3], Step [1255/1618], Loss: 2.2000, Perplexity: 9.0246batch_size:  256
embed_size:  512
Epoch [2/3], Step [1256/1618], Loss: 2.2872, Perplexity: 9.8476batch_size:  256
embed_size:  512
Epoch [2/3], Step [1257/1618], Loss: 1.9499, Perplexity: 7.0278batch_size:  256
embed_size:  512
Epoch [2/3], Step [1258/1618], Loss: 2.0509, Perplexity: 7.7750batch_size:  256
embed_size:  512
Epoch [2/3], Step [1259/1618], Loss: 2.5937, Perplexity: 13.3789batch_size:  256
embed_size:  512
Epoch [2/3], Step [1260/1618], Loss: 2.4035, Perplexity: 11.0617batch_size:  256
embed_size:  512
Epoch [2/3], Step [1261/1618], Loss: 2.2673, Perplexity: 9.6537batch_size:  256
embed_size:  512
Epoch [2/3], Step [1262/1618], Loss: 2.0726, Perplexity: 7.9452batch_size:  256
embed_size:  512
Epoch [2/3], Step [1263/1618], Loss: 2.0944, Perplexity: 8.1203batch_size:  256
embed_size:  512
Epoch [2/3], Step [1264/1618], Loss: 2.2066, Perplexity: 9.0844batch_size:  256
embed_size:  512
Epoch [2/3], Step [1265/1618

Epoch [2/3], Step [1423/1618], Loss: 2.3412, Perplexity: 10.3935batch_size:  256
embed_size:  512
Epoch [2/3], Step [1424/1618], Loss: 1.9682, Perplexity: 7.1580batch_size:  256
embed_size:  512
Epoch [2/3], Step [1425/1618], Loss: 1.9384, Perplexity: 6.9477batch_size:  256
embed_size:  512
Epoch [2/3], Step [1426/1618], Loss: 1.9813, Perplexity: 7.2520batch_size:  256
embed_size:  512
Epoch [2/3], Step [1427/1618], Loss: 1.9405, Perplexity: 6.9623batch_size:  256
embed_size:  512
Epoch [2/3], Step [1428/1618], Loss: 2.0695, Perplexity: 7.9207batch_size:  256
embed_size:  512
Epoch [2/3], Step [1429/1618], Loss: 2.1902, Perplexity: 8.9370batch_size:  256
embed_size:  512
Epoch [2/3], Step [1430/1618], Loss: 1.9484, Perplexity: 7.0177batch_size:  256
embed_size:  512
Epoch [2/3], Step [1431/1618], Loss: 2.0310, Perplexity: 7.6221batch_size:  256
embed_size:  512
Epoch [2/3], Step [1432/1618], Loss: 2.1001, Perplexity: 8.1674batch_size:  256
embed_size:  512
Epoch [2/3], Step [1433/1618]

Epoch [2/3], Step [1591/1618], Loss: 2.0558, Perplexity: 7.8134batch_size:  256
embed_size:  512
Epoch [2/3], Step [1592/1618], Loss: 2.0247, Perplexity: 7.5740batch_size:  256
embed_size:  512
Epoch [2/3], Step [1593/1618], Loss: 2.0727, Perplexity: 7.9460batch_size:  256
embed_size:  512
Epoch [2/3], Step [1594/1618], Loss: 1.9947, Perplexity: 7.3498batch_size:  256
embed_size:  512
Epoch [2/3], Step [1595/1618], Loss: 2.0357, Perplexity: 7.6578batch_size:  256
embed_size:  512
Epoch [2/3], Step [1596/1618], Loss: 1.9485, Perplexity: 7.0178batch_size:  256
embed_size:  512
Epoch [2/3], Step [1597/1618], Loss: 2.2464, Perplexity: 9.4540batch_size:  256
embed_size:  512
Epoch [2/3], Step [1598/1618], Loss: 2.0016, Perplexity: 7.4010batch_size:  256
embed_size:  512
Epoch [2/3], Step [1599/1618], Loss: 1.9836, Perplexity: 7.2692batch_size:  256
embed_size:  512
Epoch [2/3], Step [1600/1618], Loss: 2.1260, Perplexity: 8.3815
batch_size:  256
embed_size:  512
Epoch [2/3], Step [1601/1618]